In [13]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import optuna
import os
import json 


conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""
    SELECT DISTINCT
        f.match_date,
        f.team,
        f.is_home,
        f.gw,
        f.[is_promoted?],
        f.[is_early_season?],
        f.season,
        f.division,
        team_elo.elo as team_elo,
        f.team_rolling_summary_goals,
        f.team_rolling_summary_pens_made,
        f.team_rolling_summary_shots,
        f.team_rolling_summary_shots_on_target,
        f.team_rolling_summary_touches,
        f.team_rolling_summary_xg,
        f.team_rolling_summary_npxg,
        f.team_rolling_possession_touches_att_pen_area,
        f.team_rolling_keeper_psxg,
        f.team_rolling_conceded_summary_goals,
        f.team_rolling_conceded_summary_pens_made,
        f.team_rolling_conceded_summary_shots,
        f.team_rolling_conceded_summary_shots_on_target,
        f.team_rolling_conceded_summary_touches,
        f.team_rolling_conceded_summary_xg,
        f.team_rolling_conceded_summary_npxg,
        f.team_rolling_conceded_possession_touches_att_pen_area,
        f.team_rolling_conceded_keeper_psxg,
        f.opp_team,
        opp_elo.elo as opp_team_elo,
        f.opp_team_rolling_summary_goals,
        f.opp_team_rolling_summary_pens_made,
        f.opp_team_rolling_summary_shots,
        f.opp_team_rolling_summary_shots_on_target,
        f.opp_team_rolling_summary_touches,
        f.opp_team_rolling_summary_xg,
        f.opp_team_rolling_summary_npxg,
        f.opp_team_rolling_possession_touches_att_pen_area,
        f.opp_team_rolling_keeper_psxg,
        f.opp_team_rolling_conceded_summary_goals,
        f.opp_team_rolling_conceded_summary_pens_made,
        f.opp_team_rolling_conceded_summary_shots,
        f.opp_team_rolling_conceded_summary_shots_on_target,
        f.opp_team_rolling_conceded_summary_touches,
        f.opp_team_rolling_conceded_summary_xg,
        f.opp_team_rolling_conceded_summary_npxg,
        f.opp_team_rolling_conceded_possession_touches_att_pen_area,
        f.opp_team_rolling_conceded_keeper_psxg,
        ms.summary_goals as goals
    FROM 
        team_all_features_365_00050 f
    JOIN
        fbref_match_all_columns ms
            ON ms.match_url = f.match_url
            AND ms.team = f.team
    JOIN
        clubelo_features team_elo
            ON team_elo.fbref_team = f.team
                AND DATE(f.match_date) BETWEEN team_elo.start_date AND team_elo.end_date
    JOIN
        clubelo_features opp_elo
            ON opp_elo.fbref_team = f.opp_team
                AND DATE(f.match_date) BETWEEN opp_elo.start_date AND opp_elo.end_date
    WHERE 
        (f.team_rolling_summary_goals IS NOT NULL AND f.opp_team_rolling_summary_goals IS NOT NULL)
        AND f.division = 'Premier League'
                       """, conn)

df

,match_date,team,is_home,gw,is_promoted?,is_early_season?,season,division,team_elo,team_rolling_summary_goals,...,opp_team_rolling_conceded_summary_goals,opp_team_rolling_conceded_summary_pens_made,opp_team_rolling_conceded_summary_shots,opp_team_rolling_conceded_summary_shots_on_target,opp_team_rolling_conceded_summary_touches,opp_team_rolling_conceded_summary_xg,opp_team_rolling_conceded_summary_npxg,opp_team_rolling_conceded_possession_touches_att_pen_area,opp_team_rolling_conceded_keeper_psxg,goals
0,2025-05-25 00:00:00,Tottenham,1,38,0,0,2024-2025,Premier League,1778.323242,1.500408,...,1.582692,0.225854,11.154760,3.594660,595.098306,1.398281,1.220604,23.386995,1.299603,1.0
1,2025-05-25 00:00:00,Brighton,0,38,0,0,2024-2025,Premier League,1809.790894,1.706913,...,1.838188,0.046267,14.261777,4.912168,564.611159,1.755202,1.720501,26.467950,1.805191,4.0
2,2025-05-25 00:00:00,Bournemouth,1,38,0,0,2024-2025,Premier League,1806.686401,1.608621,...,2.064578,0.098667,16.699092,5.129674,671.346227,1.948015,1.838669,30.787039,1.811285,2.0
3,2025-05-25 00:00:00,Leicester City,0,38,1,0,2024-2025,Premier League,1620.328247,0.807435,...,1.334089,0.138396,13.017590,4.485684,602.876753,1.331399,1.220682,25.141040,1.460762,0.0
4,2025-05-25 00:00:00,Newcastle Utd,1,38,0,0,2024-2025,Premier League,1878.094604,1.895577,...,1.096034,0.000000,12.323471,4.252389,676.287712,1.188877,1.149149,23.893601,1.251694,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6001,2020-09-12 00:00:00,Leeds United,0,1,1,1,2020-2021,Premier League,1624.624268,1.870282,...,1.046521,0.054076,8.260537,2.720756,503.278277,1.071970,1.028709,13.422932,0.852680,3.0
6002,2020-09-12 00:00:00,Crystal Palace,1,1,0,1,2020-2021,Premier League,1673.688843,0.745718,...,1.214089,0.046936,10.890138,3.653199,580.742946,1.291557,1.246146,21.367030,1.307413,1.0
6003,2020-09-12 00:00:00,Southampton,0,1,0,1,2020-2021,Premier League,1720.211182,1.637619,...,1.610115,0.000000,13.603900,4.690449,668.948540,1.543369,1.543369,23.537592,1.642488,0.0
6004,2020-09-12 00:00:00,Fulham,1,1,1,1,2020-2021,Premier League,1591.546265,1.293899,...,1.155487,0.127042,13.836949,4.728987,568.011264,1.511493,1.394723,23.983456,1.269339,0.0


In [14]:
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=["team", "opp_team", "goals", "match_date"])
X = pd.get_dummies(X, columns=["season", "division"], drop_first=True)

y = df["goals"]

# Use matches after a certain date as test
cutoff_date = '2024-08-01'
df['match_date'] = pd.to_datetime(df['match_date'])

train_mask = df['match_date'] < cutoff_date
test_mask = df['match_date'] >= cutoff_date

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
from sklearn.linear_model import PoissonRegressor
import numpy as np
from scipy.stats import poisson
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Fit Poisson regression
print("\nTraining Poisson Regression baseline...")
poisson_reg = PoissonRegressor(
    alpha=1.0,          # Regularization strength
    max_iter=1000,      # Increase if convergence issues
    fit_intercept=True
)

poisson_reg.fit(X_train_scaled, y_train)

# Make predictions
preds_poisson = poisson_reg.predict(X_test_scaled)

# Calculate traditional metrics
rmse_poisson = np.sqrt(mean_squared_error(y_test, preds_poisson))
mae_poisson = mean_absolute_error(y_test, preds_poisson)

# Calculate log-likelihood
def poisson_log_likelihood(y_true, y_pred):
    """
    Calculate log-likelihood for Poisson distribution
    y_true: actual goal counts
    y_pred: predicted lambda (rate parameter)
    """
    # Avoid log(0) by adding small epsilon
    y_pred = np.maximum(y_pred, 1e-10)
    
    # Poisson log-likelihood: y*log(λ) - λ - log(y!)
    log_likelihood = np.sum(
        y_true * np.log(y_pred) - y_pred - np.log(np.maximum(1, np.arange(1, len(y_true) + 1)))
    )
    return log_likelihood

def poisson_log_likelihood_scipy(y_true, y_pred):
    """
    Alternative using scipy.stats - more robust
    """
    y_pred = np.maximum(y_pred, 1e-10)
    return np.sum(poisson.logpmf(y_true, y_pred))

# Calculate Poisson deviance
def poisson_deviance(y_true, y_pred):
    """
    Poisson deviance - lower is better
    """
    y_pred = np.maximum(y_pred, 1e-10)
    y_true_safe = np.maximum(y_true, 1e-10)
    
    deviance = 2 * np.sum(
        y_true * np.log(y_true_safe / y_pred) - (y_true - y_pred)
    )
    return deviance

# Calculate all metrics
log_likelihood = poisson_log_likelihood_scipy(y_test, preds_poisson)
deviance = poisson_deviance(y_test, preds_poisson)

# Mean log-likelihood (for interpretability)
mean_log_likelihood = log_likelihood / len(y_test)

print(f"\n=== POISSON REGRESSION BASELINE ===")
print(f"RMSE: {rmse_poisson:.4f}")
print(f"MAE: {mae_poisson:.4f}")
print(f"Log-likelihood: {log_likelihood:.2f}")
print(f"Mean log-likelihood: {mean_log_likelihood:.4f}")
print(f"Poisson deviance: {deviance:.2f}")
print(f"Mean prediction: {preds_poisson.mean():.2f}")
print(f"Mean actual: {y_test.mean():.2f}")

# Additional diagnostic: AIC (Akaike Information Criterion)
n_params = X_train_scaled.shape[1] + 1  # features + intercept
aic = 2 * n_params - 2 * log_likelihood
print(f"AIC: {aic:.2f}")

# Calibration check - compare predicted vs observed rates in bins
def calibration_analysis(y_true, y_pred, n_bins=10):
    """
    Check if predicted rates match observed rates across different ranges
    """
    # Create bins based on predicted values
    bin_edges = np.percentile(y_pred, np.linspace(0, 100, n_bins + 1))
    bin_indices = np.digitize(y_pred, bin_edges) - 1
    bin_indices = np.clip(bin_indices, 0, n_bins - 1)
    
    print(f"\n=== CALIBRATION ANALYSIS ===")
    print("Bin | Predicted | Observed | Count")
    print("-" * 35)
    
    for i in range(n_bins):
        mask = bin_indices == i
        if np.sum(mask) > 0:
            pred_mean = y_pred[mask].mean()
            obs_mean = y_true[mask].mean()
            count = np.sum(mask)
            print(f"{i+1:2d}  | {pred_mean:8.3f} | {obs_mean:7.3f} | {count:4d}")

# Run calibration analysis
calibration_analysis(y_test, preds_poisson)


Training Poisson Regression baseline...

=== POISSON REGRESSION BASELINE ===
RMSE: 1.1743
MAE: 0.9355
Log-likelihood: -1129.61
Mean log-likelihood: -1.4824
Poisson deviance: 820.18
Mean prediction: 1.55
Mean actual: 1.47
AIC: 2359.22

=== CALIBRATION ANALYSIS ===
Bin | Predicted | Observed | Count
-----------------------------------
 1  |    0.874 |   0.831 |   77
 2  |    1.084 |   0.974 |   76
 3  |    1.225 |   1.105 |   76
 4  |    1.324 |   1.395 |   76
 5  |    1.416 |   1.421 |   76
 6  |    1.523 |   1.382 |   76
 7  |    1.645 |   1.632 |   76
 8  |    1.790 |   1.776 |   76
 9  |    2.012 |   2.039 |   76
10  |    2.555 |   2.143 |   77


In [16]:
def calculate_team_strengths(df, model, scaler, X_train_cols, original_df):
    """
    Calculate team strengths by comparing each team against a median team at neutral venue.
    Returns goals_scored, goals_conceded, and goal_diff for each team.
    """
    
    # Get the most recent data for each team
    df_sorted = original_df.sort_values(['team', 'match_date'])
    latest_team_data = df_sorted.groupby('team').last().reset_index()
    
    print(f"Calculating strengths for {len(latest_team_data)} teams")
    print(f"Using most recent data from: {latest_team_data['match_date'].min()} to {latest_team_data['match_date'].max()}")
    
    # Calculate median values for original numeric features (before dummies)
    # Exclude categorical columns that will become dummies
    original_numeric_features = [col for col in original_df.columns 
                                if col not in ['team', 'opp_team', 'match_date', 'goals', 'season', 'division']]
    median_features = original_df[original_numeric_features].median()
    
    # For categorical variables, use the most common values from original data
    categorical_modes = {}
    if 'season' in original_df.columns:
        categorical_modes['season'] = original_df['season'].mode().iloc[0]
    if 'division' in original_df.columns:
        categorical_modes['division'] = original_df['division'].mode().iloc[0]
    
    print(f"\nMedian team stats (key features):")
    key_features = ['team_elo', 'team_rolling_summary_xg', 'team_rolling_summary_goals', 
                   'team_rolling_conceded_summary_goals', 'team_rolling_conceded_summary_xg']
    for feat in key_features:
        if feat in median_features:
            print(f"  {feat}: {median_features[feat]:.3f}")
    
    team_strengths = []
    
    for _, team_row in latest_team_data.iterrows():
        team_name = team_row['team']
        
        # === ATTACK STRENGTH: Team vs Median Defense (Neutral Venue) ===
        attack_data = team_row.copy()
        attack_data['is_home'] = 0.5  # Neutral venue (average of home/away)
        attack_data['opp_team'] = 'median_team'
        
        # Set categorical variables to mode values
        for cat_var, mode_val in categorical_modes.items():
            if cat_var in attack_data:
                attack_data[cat_var] = mode_val
        
        # Set opponent features to median values
        for feat in original_numeric_features:
            if feat.startswith('opp_team_'):
                base_feat = feat.replace('opp_team_', 'team_')
                if base_feat in median_features:
                    attack_data[feat] = median_features[base_feat]
            elif feat == 'opp_team_elo':
                attack_data['opp_team_elo'] = median_features['team_elo']
        
        # === DEFENSE STRENGTH: Median Team vs This Team's Defense (Neutral Venue) ===
        defense_data = team_row.copy()
        defense_data['is_home'] = 0.5  # Neutral venue
        defense_data['team'] = 'median_team'
        defense_data['opp_team'] = team_name
        
        # Set categorical variables to mode values
        for cat_var, mode_val in categorical_modes.items():
            if cat_var in defense_data:
                defense_data[cat_var] = mode_val
        
        # Swap team and opponent features for defense calculation
        temp_data = defense_data.copy()
        for feat in original_numeric_features:
            if feat.startswith('team_') and not feat.startswith('team_rolling_conceded'):
                opp_feat = feat.replace('team_', 'opp_team_')
                # Set attacking team (median) features
                if feat in median_features:
                    defense_data[feat] = median_features[feat]
                # Move defending team's features to opponent position
                if opp_feat in temp_data:
                    defense_data[opp_feat] = temp_data[feat]
        
        # Set ELO values correctly
        defense_data['team_elo'] = median_features['team_elo']
        defense_data['opp_team_elo'] = team_row['team_elo']
        
        # Make predictions for both scenarios
        scenarios = {
            'attack': attack_data,
            'defense': defense_data
        }
        
        predictions = {}
        
        for scenario_name, scenario_data in scenarios.items():
            # Prepare features same way as training data
            scenario_features = scenario_data.drop(['team', 'opp_team', 'goals', 'match_date'] if 'goals' in scenario_data else ['team', 'opp_team', 'match_date'])
            
            # Convert to DataFrame if it's a Series
            if isinstance(scenario_features, pd.Series):
                scenario_features = scenario_features.to_frame().T
            
            # Handle categorical variables - create dummies, but be careful about categories
            if 'season' in scenario_features.columns and 'division' in scenario_features.columns:
                scenario_features = pd.get_dummies(scenario_features, columns=["season", "division"], drop_first=True)
            elif 'season' in scenario_features.columns:
                scenario_features = pd.get_dummies(scenario_features, columns=["season"], drop_first=True)
            elif 'division' in scenario_features.columns:
                scenario_features = pd.get_dummies(scenario_features, columns=["division"], drop_first=True)
            
            # Create a DataFrame with all training columns initialized to 0
            aligned_features = pd.DataFrame(0, index=[0], columns=X_train_cols)
            
            # Fill in the values we have
            for col in scenario_features.columns:
                if col in aligned_features.columns:
                    aligned_features[col] = scenario_features[col].iloc[0]
            
            # Scale and predict
            scenario_scaled = scaler.transform(aligned_features.values)
            pred = model.predict(scenario_scaled)[0]
            predictions[scenario_name] = pred
        
        # Extract results
        goals_scored = predictions['attack']  # Goals this team scores vs median defense
        goals_conceded = predictions['defense']  # Goals median team scores vs this team's defense
        goal_diff = goals_scored - goals_conceded
        
        team_strengths.append({
            'team': team_name,
            'goals_scored': goals_scored,
            'goals_conceded': goals_conceded,
            'goal_diff': goal_diff,
            'current_elo': team_row['team_elo'],
            'latest_match_date': team_row['match_date']
        })
    
    return pd.DataFrame(team_strengths)

# Calculate team strengths
print("\nCalculating team strengths using trained Poisson model...")
team_strength_df = calculate_team_strengths(X, poisson_reg, scaler, X_train.columns.tolist(), df)

# Sort by goal difference (overall strength)
team_strength_df = team_strength_df.sort_values('goal_diff', ascending=False)

print(f"\n=== TEAM STRENGTH RANKINGS (vs Median Opponent at Neutral Venue) ===")
print("Rank | Team                | Goals Scored | Goals Conceded | Goal Diff | ELO")
print("-" * 80)
for i, (_, row) in enumerate(team_strength_df.iterrows()):
    print(f"{i+1:4d} | {row['team']:18s} | {row['goals_scored']:12.3f} | {row['goals_conceded']:14.3f} | {row['goal_diff']:9.3f} | {row['current_elo']:4.0f}")

print(f"\n=== ATTACK RANKINGS ===")
attack_sorted = team_strength_df.sort_values('goals_scored', ascending=False)
print("Rank | Team                | Goals Scored")
print("-" * 40)
for i, (_, row) in enumerate(attack_sorted.iterrows()):
    print(f"{i+1:4d} | {row['team']:18s} | {row['goals_scored']:12.3f}")

print(f"\n=== DEFENSE RANKINGS (Lower = Better) ===")
defense_sorted = team_strength_df.sort_values('goals_conceded', ascending=True)
print("Rank | Team                | Goals Conceded")
print("-" * 43)
for i, (_, row) in enumerate(defense_sorted.iterrows()):
    print(f"{i+1:4d} | {row['team']:18s} | {row['goals_conceded']:14.3f}")

# Save team strengths for future use
team_strength_df[['team', 'goals_scored', 'goals_conceded', 'goal_diff']].to_csv('team_strengths.csv', index=False)
print(f"\nTeam strengths saved to 'team_strengths.csv'")

print(f"\n=== SUMMARY STATISTICS ===")
print(f"League average goals per game: {y.mean():.3f}")
print(f"Goals scored range: {team_strength_df['goals_scored'].min():.3f} to {team_strength_df['goals_scored'].max():.3f}")
print(f"Goals conceded range: {team_strength_df['goals_conceded'].min():.3f} to {team_strength_df['goals_conceded'].max():.3f}")
print(f"Goal difference range: {team_strength_df['goal_diff'].min():.3f} to {team_strength_df['goal_diff'].max():.3f}")


Calculating team strengths using trained Poisson model...
Calculating strengths for 31 teams
Using most recent data from: 2018-05-13 00:00:00 to 2025-05-25 00:00:00

Median team stats (key features):
  team_elo: 1737.236
  team_rolling_summary_xg: 1.356
  team_rolling_summary_goals: 1.375
  team_rolling_conceded_summary_goals: 1.365
  team_rolling_conceded_summary_xg: 1.377


c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScal


=== TEAM STRENGTH RANKINGS (vs Median Opponent at Neutral Venue) ===
Rank | Team                | Goals Scored | Goals Conceded | Goal Diff | ELO
--------------------------------------------------------------------------------
   1 | Liverpool          |        2.197 |          1.138 |     1.058 | 1997
   2 | Manchester City    |        1.915 |          1.200 |     0.716 | 1945
   3 | Newcastle Utd      |        1.680 |          1.195 |     0.485 | 1878
   4 | Chelsea            |        1.786 |          1.330 |     0.456 | 1885
   5 | Crystal Palace     |        1.545 |          1.216 |     0.329 | 1820
   6 | Aston Villa        |        1.600 |          1.299 |     0.301 | 1883
   7 | Brentford          |        1.490 |          1.280 |     0.210 | 1810
   8 | Fulham             |        1.456 |          1.248 |     0.209 | 1787
   9 | Arsenal            |        1.840 |          1.632 |     0.208 | 1990
  10 | Brighton           |        1.638 |          1.471 |     0.167 | 1810
  

c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScal